In [6]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# PromptTemplate == string from template
# ChatPromptTemplate == message from template

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # temperature == for creating and randomizing the texts
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?",
                                        )

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

# a = llm.predict("How many plants are there?")

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [10]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# HumanMessage == We send a message to the AI
# AIMessage == The AI sends a message to us
# SystemMessage == We send a settings to the LLM, but it's not a question

template = ChatPromptTemplate.from_messages([
    # SystemMessage(content="You are a geography expert. And you only reply in {language}."),
    # AIMessage(content="안녕하세요. 제 이름은 {name}입니다."),
    # HumanMessage(content="What is the distance between {country_a} and {country_b}? Also what is your name?")
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕하세요. 제 이름은 {name}입니다."),
    ("human", "What is the distance between {country_a} and {country_b}? Also what is your name?")  
])
# prompt_template == The communication template for the user and the LLMS
# So prompt is very important for the LLMs answering quaility 
prompt = template.format_messages(
    language="Korean",
    name="Jin",
    country_a="Mexico",
    country_b="Thailand"
)
chat.predict_messages(prompt)


AIMessage(content='안녕하세요! 저의 이름은 Jin입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

In [14]:
# OutputParser == The communication parser for the user and the LLMs, LLM의 응답을 변형할 때가 있기 떄문에 필요하다. 
# 텍스트를 다른 자료구조로 변형하기

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()

p.parse("Hello, how, are, your")

# LangChain expression language == 

['Hello', 'how', 'are', 'your']

In [20]:
template = ChatPromptTemplate.from_messages([
    (
        "system", 
        "You are a list generating machine. Everything you are asked will be answered with a comma sepaerated list of max {max_items} in lowercase. Do NOT reply with anything else.", 
    ),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [21]:
# chat model, outputParser, template 
# create chain 

chain = template | chat | CommaOutputParser()
# .format_messages + chat.predict + .parse

# all = chain_one | chain_two | outputParser == langChain expression language

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']